In [1]:
/* Node essential patterns
*
*  The callback pattern

   the result is propagated by passing it to another function (the callback), instead
   of directly returning it to the caller
*/

// direct style
function addDirect (a, b) {
    return a + b;
}

// CPS (Continue passing style)
// it will return a value only when the callback completes its execution
function add (a, b, cb) {
    cb(a + b)
}

console.log("before");
add(2, 5, result => console.log(`the result ${result}`));
console.log("after");

before
the result 7
after


In [5]:
// Asynchronus CPS
// it will not wait for the callback to
// be executed, but instead, it returns immediately, giving the control back to
// addAsync() , and then back to its caller
function addAsync(a, b, cb) {
    setTimeout(() => cb(a + b), 100);
}

console.log("before");
addAsync(2, 5, result => console.log(`the result ${result}`));
console.log("after");

/*
it gives control back to the event loop as soon as an asynchronous request is sent, thus allowing a
new event from the queue to be processed.
*/

before
after
the result 7


In [6]:
// not every callback is CPS
const result = [1, 5, 7, 9].map(e => e -1);
console.log(result)
// the callback is used just to iterate over the elements of the array
// and not to pass the result of the operation

[ 0, 4, 6, 8 ]


In [1]:
// -*- ANTIPATTERN ALERT -*-
// dangerous patter mix syles sync / async in the same API call:
const fs = require("fs");
const cache = {};

function inconsistentRead(file, cb) {
    if (cache[file]) {
        // sync
        cb(cache[file])
    } else {
        // async
        fs.readFile(file, "utf8", (err, data) => {
            cache[file] = data;
            cb(data);
        })
    }
    
}

// -*- ANTIPATTERN ALERT -*-
// unleashing zalgo
// this will break the code
function createFileReader(file) {
    const listeners = [];
    inconsistentRead(file, value => {
       listeners.forEach(listener => listener.push(listener)); 
    });
    
    return {
        onDataReady: listener => listeners.push(listener)
    }
}


In [ ]:
// PATERN -> for sync functions prefer direct style
// direct style

// const fs = require("fs");
// const cache = {};
function consistentReadSync(file) {
    if (cache[file]) {
        return cache[file]
    } else {
        cache[file] =fs.readFileSync(file, "utf8");
        return cache[file]
    }
}

In [1]:
// deferred execution
// const fs = require("fs");
// process.nextTick
// const cache = {};
/*
We guarantee that a callback is invoked asynchronously by deferring its execution using process.nextTick()
*/

function consistentReadAsync(file, cb) {
    if(cache[file]) {
        process.nextTick(() => cb(cache[file]))
    } else {
        // async call
        fs.readFile(file, "utf8", (err, data) => {
            cache[file] = data;
            cb(data);
        });
    }    
}

/*
Callbacks deferred with process.nextTick() run before any other I/O event is fired, while with
setImmediate() , the execution is queued behind any I/O event that is already in the queue
*/